# Automatic Differentiation

Optimizasyon işlemi için elde türev almıştık önceki derste basit bir neural network'te. İlk iterasyon için forward ve backward hesabı elle yapabiliriz, ama binlerde hatta yüzbinlerce öznitelik vektörü için, binlerce nöronu olan çok fazla hidden layerlı bir neural network için bu türevleri, ileri geri işlemleri hesaplayabilir miyiz?

Hayır!

Derin öğrenme yöntemlerinde sistem otomatik olarak bir *computational graph* oluşturuyor, ve hangi veri hangi operasyonla ilintili bunların kaydını tutuyor.

Bu sayede, backpropagation yapmak hafızadaki bilgileri kullanarak mümkün oluyor.

Backpropagation sürecinde sistem, oluşturduğu computational graph üzerinde geri gidiyor, ve her parametrenin birbirine göre türevini alarak sonuçları hesaplıyor.


## Basit Örnek

Neural network'de elimizdeki fonksiyon:
$y = 2\mathbf{x}^{\top}\mathbf{x}$

x skalar değerleri için y'nin türevlerinin sonuçları ne olacak? Onları hesaplayalım!


In [ ]:
import torch

# x'e skalar değerler ver, içeriği fark etmez
x = torch.arange(4.0)
x

tensor([0., 1., 2., 3.])

Türev hesabından önce yapılması gereken önemli bir adım:

x değerlerini türeve hazır şekilde hazıfaya kaydetmek!

Her seferinde tekrar tekrar değerler bulup hafızaya yazarsak, RAM dolar ve hata alırız. Yani değerler hafızada sabit bir yerde kalmalı. 

In [ ]:
# Hafızada türeve hazır halde sabitle
x.requires_grad_(True)  # Alternatif olarak `x = torch.arange(4.0, requires_grad=True)`

tensor([0., 1., 2., 3.], requires_grad=True)

In [ ]:
# Hiçbir şeyin x'e göre türev sonucu ne? Hafızada ona dair bir fonksiyon tanımlamadık henüz.

print(x.grad)

None


y fonksiyonunu yazalım


In [ ]:
y = 2 * torch.dot(x, x)
y

tensor(28., grad_fn=<MulBackward0>)

y'nin x'e göre türevini nasıl hesaplarız elde hesap yapmadan?

backpropagation


In [ ]:
y.backward()

türev sonuçları hafızada saklı:

In [ ]:
x

tensor([0., 1., 2., 3.], requires_grad=True)

In [ ]:
# elde hesaplasak 4*x'e denk gelecek bu türev
x.grad

tensor([ 0.,  4.,  8., 12.])

In [ ]:
x.grad == 4 * x

tensor([True, True, True, True])

x'e bağlı başka bir fonksiyon hesabı yapacaksak, x türev bilgilerini hafızadan temizlememiz gerekiyor. Yoksa her tekrarda ikinci üçüncü derece gradient almaya kalkar önceki değerlerin üzerine


In [ ]:
# PyTorch accumulates the gradient in default, we need to clear the previous
# values
x.grad.zero_()
x.grad

tensor([0., 0., 0., 0.])

## Skalar olmayan değerler için?

Gerçek uygulamalarda veriler tek tek değil, batch halinde verilir. Bu batch'ler için türev nasıl hesaplanacak?

İş artık vektörden matrixe dönüşecek

Partial derivative'lerin toplamı lazım


In [ ]:
x

tensor([0., 1., 2., 3.], requires_grad=True)

In [ ]:
# öncelikle x'in hafızadaki türevlerini sıfırla
x.grad.zero_()

# y fonksiyonumuz
y = x * x
print(y)

# y.backward(torch.ones(len(x))) equivalent to the below
y.sum().backward()
x.grad

tensor([0., 1., 4., 9.], grad_fn=<MulBackward0>)


tensor([0., 2., 4., 6.])

## Hafızadakini bozmadan işlem yapabilmek

Bazı hesapları hafızadakinin dışında yapmak isteriz (computational graph dışında).

Mesela, `y`, `x`e bağlı bir fonksiyon ve bunu hesapladık diyelim.
Daha sonra `z` de `y` ve `x`e bağlı bir fonksiyon olarak tanımlandı:

z = y*x

(output layer'daki fonksiyonu hatırlayın, hem bir önceki layer aktivasyon fonksiyonuna hem de x'e bağlıydı). Yani z de önemli

`z`nin `x`e göre türevini nasıl hesaplarız?

`y`yi hafızadan ayıralım, y'nin x'den hesaplanan değerini u variable'ına kaydedelim. u'da değer olsun ama y'nin computational graph'ta nasıl hesaplandığı bilgisi olmasın (yani hafızadaki yerinden farklı yerde olsun).

Yani z'nin türevini hesaplarken y kısmı x'e kadar gitmesin, ara sonuç u bize lazım.

Böylece backpropagation'da `z = u * x`un x'e göre türevini hesaplarken u değerini sabit alacağız `z = x * x * x`'in türevini almak yerine


In [ ]:
x.grad.zero_()
y = x * x
u = y.detach() # u'yu y'nin hafızadaki yerinden başka bir yere kopyalayıp içinde sadece y'nin skalar değerlerini tutuyoruz
print(u)
z = u * x

print(z)

tensor([0., 1., 4., 9.])
tensor([ 0.,  1.,  8., 27.], grad_fn=<MulBackward0>)


In [ ]:
z.sum().backward()

print(x.grad)

x.grad == u

tensor([0., 1., 4., 9.])


tensor([True, True, True, True])

Şimdi y'nin x'e göre türevini hesaplayalım:

 `y = x * x` in `x`e göre türevi `2 * x` olacak


In [ ]:
x.grad.zero_()
y.sum().backward()
x.grad == 2 * x

tensor([True, True, True, True])

## Çılgın fonksiyonların bile türevi hesaplanabilir


In [ ]:
def f(a):
    b = a * 2
    while b.norm() < 1000:
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c

In [ ]:
# Türevi

a = torch.randn(size=(), requires_grad=True) #türevlenebilir oluşturuyoruz

print(a)

d = f(a)

d.backward()

print(d)

tensor(0.3407, requires_grad=True)
tensor(1395.4294, grad_fn=<MulBackward0>)


Fonksiyona bakınca `f(a) = k * a`, değer `k` ya bağlı. d = f(a) olunca türev de k oluyor.

Doğal olarak türevin `d / a = k` olması normal.


In [ ]:
a.grad == d / a

tensor(True)